In [37]:
from utils import * 
import pennylane as qml
import os 
from autoencoder8 import *
fid_fold = 'out_fi'
emd_fold = 'out_em'



In [38]:
binder_fi = {}

for subfolder in os.listdir(fid_fold):
    mq = int(subfolder[4])-int(subfolder[-1])
    subfolder_path = os.path.join(fid_fold, subfolder)
    binder_fi[mq]=[]
    for batchfolder in os.listdir(subfolder_path):
        batchfolder_path = os.path.join(subfolder_path, batchfolder)
        for file in os.listdir(batchfolder_path):
            file_path= os.path.join(batchfolder_path, file)
            if 'weights' in file:
                weight_file = os.path.join(batchfolder_path, file)
                append = file[7:-3]
                loss_file = os.path.join(batchfolder_path, f'loss_train_{append}txt')
                binder_fi[mq].append((weight_file, loss_file))

In [39]:
binder_em = {}

for subfolder in os.listdir(emd_fold):
    mq = int(subfolder[4])-int(subfolder[-1])
    subfolder_path = os.path.join(emd_fold, subfolder)
    binder_em[mq]=[]
    for batchfolder in os.listdir(subfolder_path):
        batchfolder_path = os.path.join(subfolder_path, batchfolder)
        for file in os.listdir(batchfolder_path):
            file_path= os.path.join(batchfolder_path, file)
            if 'weights' in file:
                weight_file = os.path.join(batchfolder_path, file)
                append = file[7:-3]
                tloss_file = os.path.join(batchfolder_path, f'loss_train_{append}npy')
                vloss_file = os.path.join(batchfolder_path, f'loss_val{append}npy')
                binder_em[mq].append((weight_file, tloss_file,vloss_file,file))

In [40]:
data = get_data(8)
X=[data['ground_states'][x] for x in range(100)]


In [ ]:
res = pd.DataFrame(columns=['vloss','tloss','avg_fidelity','file','train_type','mq','data_idx'])
for mq in range(7,8):
    for model in binder_em[mq]:
        n_qubit = 8
        t_qubit = n_qubit-mq
        dvc = qml.device('default.mixed', wires=n_qubit+t_qubit, shots=None)
        @qml.qnode(dvc)
        def circ(ae,dm1):
            qml.StatePrep(dm1, wires=range(0,n_qubit))
            wire_map=dict(zip(list(range(t_qubit)),list(range(n_qubit,n_qubit+t_qubit))))
            ae.get_cirq(0)
            qml.adjoint(qml.map_wires(ae.get_cirq, wire_map))(0)
            return qml.density_matrix(list(range(t_qubit, len(dvc.wires)+1)))

        dvc = qml.device('default.mixed', wires=n_qubit, shots=None)
        ae = Axutoencoder(n_qubit,t_qubit,dvc,'c11')
        ae.set_layers(3)
        ae.set_weights_loss(np.load(model[0]), np.load(model[1]))
        # qml.draw_mpl(circ)(ae,X[0])
        # fidel = []
        for i in range(2):
            # fidel.append(qml.math.fidelity( circ(ae,X[i]),np.outer(X[i], np.conj(X[i]))))
            fidel= qml.math.fidelity( circ(ae,X[i]),np.outer(X[i], np.conj(X[i])))
            timeindex = np.argmin(np.load(model[2]))+1
            vloss = np.min(np.load(model[2]))
            tloss = np.min(np.load(model[1]))
            d = {'vloss':vloss,
                'tloss':tloss,
                'avg_fidelity':fidel,
                'file':model[3],
                'train_type':'em',
                'mq':mq,
                'data_idx':i}
            res= pd.concat([res,pd.DataFrame([d])])


C:\Users\orazi\AppData\Local\Temp\ipykernel_5088\814034858.py:33: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  res= pd.concat([res,pd.DataFrame([d])])


In [55]:
res

,vloss,tloss,avg_fidelity,file,train_type,mq
0,0.019413305354073873,0.026890170747675485,0.989703,weights(8)(7)_0.npy,em,7
0,0.019413305354073873,0.026890170747675485,0.990241,weights(8)(7)_0.npy,em,7
0,0.027472834625169796,0.044276449112453195,0.862604,weights(8)(7)_1.npy,em,7
0,0.027472834625169796,0.044276449112453195,0.832103,weights(8)(7)_1.npy,em,7
0,0.03904099787961787,0.04243187851035269,0.890530,weights(8)(7)_2.npy,em,7
0,0.03904099787961787,0.04243187851035269,0.873180,weights(8)(7)_2.npy,em,7
0,0.018691219573071204,0.02812726395098612,0.991883,weights(8)(7)_3.npy,em,7
0,0.018691219573071204,0.02812726395098612,0.993485,weights(8)(7)_3.npy,em,7
0,0.02820138489464378,0.02943678056778349,0.991131,weights(8)(7)_0.npy,em,7
0,0.02820138489464378,0.02943678056778349,0.993937,weights(8)(7)_0.npy,em,7


In [42]:
binder_fi

{7: [('out_fi\\out_8to1\\10\\weights(8)(7)_0.npy',
   'out_fi\\out_8to1\\10\\loss_train_(8)(7)_0.txt'),
  ('out_fi\\out_8to1\\10\\weights(8)(7)_1.npy',
   'out_fi\\out_8to1\\10\\loss_train_(8)(7)_1.txt'),
  ('out_fi\\out_8to1\\10\\weights(8)(7)_2.npy',
   'out_fi\\out_8to1\\10\\loss_train_(8)(7)_2.txt'),
  ('out_fi\\out_8to1\\10\\weights(8)(7)_3.npy',
   'out_fi\\out_8to1\\10\\loss_train_(8)(7)_3.txt'),
  ('out_fi\\out_8to1\\5\\weights(8)(7)_0.npy',
   'out_fi\\out_8to1\\5\\loss_train_(8)(7)_0.txt'),
  ('out_fi\\out_8to1\\5\\weights(8)(7)_1.npy',
   'out_fi\\out_8to1\\5\\loss_train_(8)(7)_1.txt')],
 6: [('out_fi\\out_8to2\\10\\weights(8)(6)_0.npy',
   'out_fi\\out_8to2\\10\\loss_train_(8)(6)_0.txt'),
  ('out_fi\\out_8to2\\10\\weights(8)(6)_1.npy',
   'out_fi\\out_8to2\\10\\loss_train_(8)(6)_1.txt'),
  ('out_fi\\out_8to2\\10\\weights(8)(6)_2.npy',
   'out_fi\\out_8to2\\10\\loss_train_(8)(6)_2.txt'),
  ('out_fi\\out_8to2\\10\\weights(8)(6)_3.npy',
   'out_fi\\out_8to2\\10\\loss_train_(8)